<a href="https://colab.research.google.com/github/dp2292/CP01-Data-Alchemists/blob/Dhruv/SBT_Graphs_Dhruv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! gdown --id 1RfhNYBd7g70LdgJROvx8iPGygPhiGQLZ

/usr/local/lib/python3.10/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1RfhNYBd7g70LdgJROvx8iPGygPhiGQLZ
To: /content/CP01.zip
100% 630M/630M [00:04<00:00, 136MB/s]


In [ ]:
!unzip /content/CP01.zip

Archive:  /content/CP01.zip
  inflating: For_modeling.csv        
  inflating: cleaned_seoul_bike_data.csv  
